In [ ]:
import os
import cv2
import yaml
import glob
import random
import albumentations as A
from tqdm import tqdm
from ultralytics import YOLO
import glob

os.environ["TENSORBOARD_BINARY"] = "false"
os.environ["WANDB_DISABLED"] = "true"

DATASET_PATH = "/kaggle/input/vw-logo-dataset/MKR_datadet"

train_img_dir = f"{DATASET_PATH}/train/images"
train_lbl_dir = f"{DATASET_PATH}/train/labels"

test_img_dir = f"{DATASET_PATH}/test/images"
test_lbl_dir = f"{DATASET_PATH}/test/labels"

os.makedirs("augmented/train/images", exist_ok=True)
os.makedirs("augmented/train/labels", exist_ok=True)

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.RandomScale(scale_limit=0.2, p=0.4),
    A.Perspective(scale=(0.02, 0.05), p=0.3),
    A.RandomBrightnessContrast(p=0.4),
    A.HueSaturationValue(p=0.3),
    A.MotionBlur(blur_limit=5, p=0.2),
    A.Blur(blur_limit=3, p=0.2),
    A.RandomShadow(p=0.1),
    A.RandomRain(p=0.15),
    A.RandomFog(p=0.1),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))



train_images = glob.glob(train_img_dir + "/*.jpg") + \
               glob.glob(train_img_dir + "/*.png") + \
               glob.glob(train_img_dir + "/*.jpeg")

for img_path in tqdm(train_images, desc="Augmenting"):
    filename = os.path.basename(img_path).split('.')[0]

    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # читаємо YOLO мітки
    label_path = os.path.join(train_lbl_dir, filename + ".txt")
    with open(label_path, "r") as f:
        labels = f.read().splitlines()

    bboxes = []
    classes = []
    for line in labels:
        cls, xc, yc, bw, bh = map(float, line.split())
        bboxes.append([xc, yc, bw, bh])
        classes.append(int(cls))

    transformed = transform(image=img, bboxes=bboxes, class_labels=classes)
    aug_img = transformed['image']
    aug_boxes = transformed['bboxes']
    aug_classes = transformed['class_labels']

    out_img_path = f"augmented/train/images/{filename}_aug.jpg"
    out_lbl_path = f"augmented/train/labels/{filename}_aug.txt"

    cv2.imwrite(out_img_path, aug_img)

    with open(out_lbl_path, "w") as f:
        for cls, box in zip(aug_classes, aug_boxes):
            f.write(f"{cls} " + " ".join(map(str, box)) + "\n")

data_yaml = {
    'train': os.path.abspath("augmented/train/images"),
    'val': test_img_dir,
    'nc': 1,
    'names': ['VW_logo']
}

with open('data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)


model = YOLO('yolo11n.pt')
model.train(
    data='data.yaml',
    epochs=250,
    imgsz=640,
    batch=8,
    workers=2,
    plots=False,
    verbose=True,
)

input_video = "/kaggle/input/vw-video-logo/VW_tiguan.mp4"
output_video = "output_detected.mp4"

train_dirs = sorted(glob.glob("runs/detect/train*"))

if not train_dirs:
    raise RuntimeError("Не знайдено жодної тренувальної папки!")

last_train = train_dirs[-1]
best_path = f"{last_train}/weights/best.pt"

model = YOLO(best_path)

cap = cv2.VideoCapture(input_video)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (w, h))

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame,
        conf=0.65,
        iou=0.35,
        persist=True,
        tracker="bytetrack.yaml"
    )

    results = model(frame, conf=0.65)

    res = results[0]
    
    if res.boxes is not None:
        for box in res.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls)
            conf = float(box.conf)

            if frame_count == 1:
                continue
    
            ratio = (x2-x1) / (y2-y1)
            if not 0.75 < ratio < 1.30:
                continue
    
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, f"{conf:.2f}", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    out.write(frame)

    frame_count += 1

cap.release()
out.release()
print("Збережено як:", output_video)


Augmenting: 100%|██████████| 45/45 [00:05<00:00,  7.75it/s]

New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'


Ultralytics 8.3.23 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=250, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_wid

train: Scanning /kaggle/working/augmented/train/labels... 45 images, 0 backgrounds, 0 corrupt: 100%|██████████| 45/45 [00:00<00:00, 1282.80it/s]

train: New cache created: /kaggle/working/augmented/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all         11         12      0.995          1      0.995      0.681



250 epochs completed in 0.101 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 5.5MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.23 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 24.14it/s]

                   all         11         12      0.995          1      0.995      0.718
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.7ms postprocess per image


requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.6s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 384x640 (no detections), 52.4ms
Speed: 2.4ms preprocess, 52.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.4ms
Speed: 2.6ms preprocess, 9.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.9ms
Speed: 2.8ms preprocess, 9.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.3ms
Speed: 2.4ms preprocess, 9.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 VW_logo, 10.2ms
Speed: 2.6ms preprocess, 10.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 VW_logo, 